In [54]:
import time
import requests
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from dateutil import parser
from datetime import datetime

In [2]:
fake = faker.Faker(locale='en')

In [73]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).replace('\t', ' ').strip()

In [74]:
alibaba_headers = {
    'User-Agent': "Mozilla/5.0 (Linux; Android 8.0; Pixel 2 Build/OPD3.170816.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Mobile Safari/537.36",
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'content-type': 'application/json',
    'referer': 'https://careers.alibaba.com/positionList.htm?keyword=&location='
}

In [75]:
VAC_LIST_URL = "https://careers.alibaba.com/socialPositionList.json?_api=getPositionList&_mock=false&categorys=&keyword=&location="
VAC_DETAIL_URL = "https://careers.alibaba.com/positionDetail.json?_api=getPositionDetail&_mock=false&positionId="

In [76]:
search_params = {
  'pageSize': 1000,
  'pageIndex': 1,
  'stamp': int(time.time() * 1000)
}

In [77]:
list_result = requests.get(VAC_LIST_URL, headers=alibaba_headers, params=search_params).json()['content']
list_result.keys()

dict_keys(['datas', 'empty', 'pageIndex', 'pageSize', 'totalPage', 'totalRecord'])

In [78]:
list_result['datas'][0].keys()

dict_keys(['canReferral', 'canShare', 'canTransfer', 'categoryEnName', 'categoryId', 'categoryName', 'code', 'degree', 'departmentId', 'departmentName', 'description', 'effectiveDate', 'gmtCreate', 'gmtModified', 'id', 'isOpen', 'isSocial', 'isUrgent', 'name', 'offerRecruitsNumber', 'recruitNumber', 'recruitmentArea', 'requirement', 'status', 'uneffectualDate', 'workExperience', 'workLocation'])

In [79]:
job_dicts = []
for job_info in tqdm(list_result['datas']):
  if job_info['isOpen'] == 'Y':
    job_dict = {
      'title': clear_string(job_info['name']),
      'internal_id': job_info['id'],
      'url': VAC_DETAIL_URL + job_info['id'],
      'description': clear_string(job_info['description']),
      'responsibilities': clear_string(job_info['requirement']),
      'qualifications': None,
      'company': 'Alibaba',
      'location': clear_string(job_info['workLocation']) if job_info['workLocation'] else None,
      'department': clear_string(job_info['departmentName']) if job_info['departmentName'] else None,
      'publish_date': datetime.fromtimestamp(float(job_info["gmtCreate"]) / 1000)
    }
    job_dicts.append(job_dict)
  else:
    print(f'Skipping vac with id {job_info["id"]} because it is closed')

100%|██████████| 310/310 [00:00<00:00, 11104.10it/s]


In [80]:
snapshot = pd.DataFrame(filter(lambda x: x is not None, job_dicts))
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,location,department,publish_date
186,蚂蚁集团 NLP算法专家 Team Leader 认知计算和知识图谱,MbBtcq1JQpov8sZNYPM2Cg%3D%3D,https://careers.alibaba.com/positionDetail.jso...,1. 领导和建立世界一流的NLP算法研发团队（10人+）\n 2. 支持公司内的各类NLP应...,1. 自然语言理解、机器学习相关领域学位，曾在顶级会议发表论文（主要集中在 ACL，EMNL...,None,Alibaba,"Beijing,Beijing,China",知识图谱与自然语言处理,2022-02-15 08:23:16
128,Alibaba Group Sales-Account Executive New York,tPy7R4_I1PIfw12B%2FWl4Pw%3D%3D,https://careers.alibaba.com/positionDetail.jso...,What you'd be selling\n \n This is an opportun...,- Identify and source new partners through inb...,None,Alibaba,"New York,New York,United States",GGS 销售部,2022-05-13 06:51:09
133,Ant Group Senior Sales Manager (China-Desk) FT...,Rb4jbbQ2qiJU1AnK%2FcrZ0w%3D%3D,https://careers.alibaba.com/positionDetail.jso...,WorldFirst is shaking up the financial service...,"You are fluent in Mandarin and English, with a...",None,Alibaba,"Amsterdam,Netherlands",Sales,2022-07-14 14:43:09
109,蚂蚁集团 FIBD HK 国际事业群,dgpcOCiaFpr7ILs7Be3mhw%3D%3D,https://careers.alibaba.com/positionDetail.jso...,1. Bank cooperation： Business cooperation with...,"1. Prior work experience with banks, card sche...",None,Alibaba,Hong Kong,金融机构支持及管理,2022-04-06 12:53:20
34,Ant Group Developer Community Manager OceanBase,U7WajyrEx40i5ASxOz1FDA%3D%3D,https://careers.alibaba.com/positionDetail.jso...,1.Build and execute evangelism programs that s...,1.5+ years of experience in technical evangeli...,None,Alibaba,"Singapore,Singapore",国际业务部,2022-08-08 09:43:01


In [81]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             310 non-null    object        
 1   internal_id       310 non-null    object        
 2   url               310 non-null    object        
 3   description       310 non-null    object        
 4   responsibilities  310 non-null    object        
 5   qualifications    0 non-null      object        
 6   company           310 non-null    object        
 7   location          310 non-null    object        
 8   department        310 non-null    object        
 9   publish_date      310 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(9)
memory usage: 24.3+ KB


In [82]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'11-08-2022'

In [83]:
snapshot.to_csv(f'../data/alibaba/{current_date}.csv')
snapshot.to_excel(f'../data/alibaba/{current_date}.xlsx', engine='xlsxwriter')